In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("./data/botswana.tsv", sep="\t")

In [3]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [4]:
data[data.notnull().all(axis=1)]

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0
5,1,30,5,spirit,5.0,1.0,0.0,1,24.0,7.0,1,1.0,0.0,0.0,0.0
6,3,42,4,other,3.0,1.0,0.0,1,15.0,11.0,1,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4353,9,49,0,protestant,5.0,0.0,0.0,1,15.0,0.0,0,0.0,1.0,0.0,0.0
4354,3,31,2,protestant,4.0,1.0,1.0,1,18.0,0.0,0,0.0,1.0,0.0,0.0
4355,4,27,6,protestant,4.0,1.0,1.0,1,17.0,7.0,0,0.0,0.0,0.0,0.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0


In [5]:
data["nevermarr"] = data.agefm.apply(lambda x: 1 if np.isnan(x) else 0)

In [6]:
data.agefm.fillna(0, inplace=True)

In [7]:
data.drop("evermarr", axis=1, inplace=True)

In [8]:
data.loc[data.nevermarr == 1, "heduc"] = -1

In [9]:
sum(data.heduc.isna())

123

In [10]:
data["idlnchld_noans"] = data.idlnchld.apply(lambda x: -1 if np.isnan(x) else 0)
data["heduc_noans"] = data.heduc.apply(lambda x: -2 if np.isnan(x) else 0)
data["usemeth_noans"] = data.usemeth.apply(lambda x: -1 if np.isnan(x) else 0)

In [11]:
data.idlnchld.fillna(-1, inplace=True)
data.heduc.fillna(-2, inplace=True)
data.usemeth.fillna(-1, inplace=True)

In [12]:
data.dropna(inplace=True)

In [13]:
data.idlnchld_noans.unique() * -1

array([0, 1], dtype=int64)

In [14]:
4348 * 18

78264

In [15]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc +\
            urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=data)

In [16]:
fitted = m1.fit()

In [17]:
fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Mon, 14 Feb 2022   Prob (F-statistic):               0.00
Time:                        17:23:14   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.212     -4.835      0.000      -1.443      -0.610
religion[T.other]         -0.0830      0.083     -1.001      0.317      -0.245       0.080
religion[T.protestant]    -0.0149      0.082     -0.181      0.857      -0.176       0.146
religion[T.spirit]        -0.0191      0.077     -0.248      0.804      -0.171       0.132
age                        0.1703      0.003     51.891      0.000       0.164       0.177
educ                      -0.0724      0.007     -9.843      0.000      -0.087      -0.058
idlnchld                   0.0760      0.011      6.923      0.000       0.054       0.098
knowmeth                   0.5564      0.121      4.580      0.000       0.318       0.795
usemeth                    0.6473      0.048     13.424      0.000       0.553       0.742
agefm                     -0.0604      0.007     -9.213      0.000      -0.073      -0.048
heduc                     -0.0551      0.008     -6.838      0.000      -0.071      -0.039
urban                     -0.2137      0.047     -4.527      0.000      -0.306      -0.121
electric                  -0.2685      0.077     -3.479      0.001      -0.420      -0.117
radio                     -0.0235      0.051     -0.461      0.645      -0.123       0.076
tv                        -0.1451      0.093     -1.566      0.118      -0.327       0.037
bicycle                    0.2139      0.050      4.260      0.000       0.115       0.312
nevermarr                 -2.2393      0.148    -15.143      0.000      -2.529      -1.949
idlnchld_noans            -0.6539      0.153     -4.286      0.000      -0.953      -0.355
heduc_noans                0.4362      0.072      6.026      0.000       0.294       0.578
usemeth_noans             -0.7652      0.196     -3.910      0.000      -1.149      -0.382
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         360.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [19]:
data.drop(["radio", "religion", "tv"], inplace=True, axis=1)

In [20]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc +\
            urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=data)
fitted_2 = m2.fit()
fitted_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Mon, 14 Feb 2022   Prob (F-statistic):               0.00
Time:                        17:23:14   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.393      0.000      -1.459      -0.681
age                0.1702      0.003     52.271      0.000       0.164       0.177
educ              -0.0729      0.007    -10.285      0.000      -0.087      -0.059
idlnchld           0.0770      0.011      7.042      0.000       0.056       0.098
knowmeth           0.5610      0.121      4.628      0.000       0.323       0.799
usemeth            0.6516      0.048     13.537      0.000       0.557       0.746
agefm             -0.0606      0.007     -9.240      0.000      -0.073      -0.048
heduc             -0.0573      0.008     -7.186      0.000      -0.073      -0.042
urban             -0.2190      0.047     -4.682      0.000      -0.311      -0.127
electric          -0.3207      0.070     -4.584      0.000      -0.458      -0.184
bicycle            0.2046      0.049      4.154      0.000       0.108       0.301
nevermarr         -2.2501      0.148    -15.231      0.000      -2.540      -1.961
idlnchld_noans    -0.6565      0.152     -4.310      0.000      -0.955      -0.358
heduc_noans        0.4427      0.072      6.122      0.000       0.301       0.584
usemeth_noans     -0.7732      0.196     -3.955      0.000      -1.156      -0.390
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         344.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
print("F=%f, p=%f, k1=%f" % fitted.compare_f_test(fitted_2))

F=0.919236, p=0.467231, k1=5.000000


In [22]:
round(0.467231, 4)

0.4672

In [23]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc +\
            urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans', data=data)
fitted_3 = m3.fit()
fitted_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Mon, 14 Feb 2022   Prob (F-statistic):               0.00
Time:                        17:23:14   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.897      0.000      -1.590      -0.796
age                0.1776      0.003     54.368      0.000       0.171       0.184
educ              -0.0560      0.007     -7.862      0.000      -0.070      -0.042
idlnchld           0.0705      0.011      6.317      0.000       0.049       0.092
knowmeth           0.8739      0.121      7.203      0.000       0.636       1.112
agefm             -0.0649      0.007     -9.716      0.000      -0.078      -0.052
heduc             -0.0521      0.008     -6.411      0.000      -0.068      -0.036
urban             -0.1866      0.048     -3.917      0.000      -0.280      -0.093
electric          -0.3218      0.071     -4.505      0.000      -0.462      -0.182
bicycle            0.1979      0.050      3.935      0.000       0.099       0.296
nevermarr         -2.3625      0.150    -15.707      0.000      -2.657      -2.068
idlnchld_noans    -0.5266      0.155     -3.393      0.001      -0.831      -0.222
heduc_noans        0.3973      0.074      5.389      0.000       0.253       0.542
==============================================================================
Omnibus:                      250.641   Durbin-Watson:                   1.910
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              936.515
Skew:                          -0.158   Prob(JB):                    4.35e-204
Kurtosis:                       5.251   Cond. No.                         344.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
print("F=%f, p=%f, k1=%f" % fitted_2.compare_f_test(fitted_3))

F=92.890582, p=0.000000, k1=2.000000


In [25]:
fitted_2.compare_f_test(fitted_3)

(92.89058230109666, 3.155200948041877e-40, 2.0)

In [44]:
m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc +\
            urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=adata)
fitted_4 = m4.fit(cov_type='HC1')
fitted_4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Mon, 14 Feb 2022   Prob (F-statistic):               0.00
Time:                        17:30:54   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.152      0.000      -1.575      -0.565
age                0.1702      0.004     38.746      0.000       0.162       0.179
educ              -0.0729      0.007    -10.311      0.000      -0.087      -0.059
idlnchld           0.0770      0.014      5.323      0.000       0.049       0.105
knowmeth           0.5610      0.174      3.224      0.001       0.220       0.902
usemeth            0.6516      0.052     12.571      0.000       0.550       0.753
agefm             -0.0606      0.010     -6.192      0.000      -0.080      -0.041
heduc             -0.0573      0.009     -6.440      0.000      -0.075      -0.040
urban             -0.2190      0.045     -4.814      0.000      -0.308      -0.130
electric          -0.3207      0.063     -5.076      0.000      -0.445      -0.197
bicycle            0.2046      0.048      4.279      0.000       0.111       0.298
nevermarr         -2.2501      0.202    -11.158      0.000      -2.645      -1.855
idlnchld_noans     0.6565      0.216      3.043      0.002       0.234       1.079
heduc_noans        0.4427      0.095      4.638      0.000       0.256       0.630
usemeth_noans     -0.7732      0.212     -3.639      0.000      -1.190      -0.357
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         344.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [32]:

adata = data

adata.idlnchld_noans = data.idlnchld_noans * -1

In [35]:
adata.usemeth

0       1.0
1       1.0
2       0.0
3       0.0
4       1.0
       ... 
4356    0.0
4357    1.0
4358    1.0
4359    0.0
4360    1.0
Name: usemeth, Length: 4348, dtype: float64

In [41]:
adata.deb

0           -inf
1       0.693147
2           -inf
3           -inf
4       1.098612
          ...   
4356        -inf
4357    0.693147
4358    1.386294
4359    0.000000
4360    1.791759
Name: ceb, Length: 4348, dtype: float64